<a href="https://colab.research.google.com/github/nhoelterhoff/AIPND/blob/master/b81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# B81 Analysis
Analysis of all B81 Events for the locations Barcelona, Berlin, Cologne, Hamburg, London, Munich & Vienna.

This analysis is structured as follows:
1. Data Cleansing
2. Analysis

## 1. Data Cleansing
- We need to import the data, and filter out all empty columns. 
- Furthermore, we only would like to see all published events.

The resulting dataframe will be stored within *events_cleaned.csv*.

In [ ]:
# set up libraries
import pandas as pd
import numpy as np
from google.colab import drive
import pytz
import datetime

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/My Drive/Freelance/beat81/all_events.csv'
df_all = pd.read_csv(path)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,11,26,38,39,82,83,94,95,104,110,111,112,113,114,115,116,119,120,142,154,156,157,158) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df_all.shape

(75622, 163)

### Remove Empty Columns

In [ ]:
# remove empty columns from dataset
non_empty_columns = []
empty_columns = []
for column in list(df_all.columns):
  # if the count of all null values equals the row number, the column is empty
  if df_all[column].isnull().sum() == df_all.shape[0]:
    empty_columns.append(column)
  else:
    non_empty_columns.append(column)
    
print('empty columns:')
print(empty_columns)
print('columns with content:')
print(non_empty_columns)
# print(columns_with_nan)

empty columns:
['company', 'tags', 'subtags', 'is_successfully_tracked', 'tracking_failure_summary', 'tracking_failure_description', 'tracking_failure_source', 'tracking_failure_unit', 'flags', 'current_status.meta', 'type.seo_slug', 'type.workout_plan_id', 'type.tags', 'type.subtags', 'type.flags', 'type.name_id', 'location.slug', 'location.sys_flags', 'location.address.city', 'location.address.address2', 'location.address.district', 'location.address.city_code', 'location.address.country_code', 'location.address.district_code', 'location.map_image', 'coach.profile_picture', 'location.opening_hours', 'type.header_image_desc', 'location.address', 'sys_flags', 'current_status', 'location.feature_flags']
columns with content:
['id', 'event_type_id', 'coach_id', 'location_id', 'current_status_id', 'recurrent_id', 'is_published', 'date_begin', 'duration', 'max_participants', 'type_special', 'participants_count', 'attendees_count', 'no_shows_count', 'permitted_cancellations_count', 'illicit

In [ ]:
# create df only with columns with content
df = df_all[non_empty_columns]

### Remove Unpublished Events

In [ ]:
# only keep events that were published
published = df_all['is_published'] == True
df = df[published]

### Clean Workout Categories

In [ ]:
df['type.workout_category'].value_counts()

hiit               41581
strength-cardio    24441
strength            4420
cycling             2424
Yoga                 572
Yoga-HIIT            442
beginner             423
online               256
mobility               8
Hiit                   3
Strength               1
Name: type.workout_category, dtype: int64

In [ ]:
df['type.workout_category'].replace('Hiit','hiit', inplace=True)

In [ ]:
df['type.workout_category'].replace('Strength ','strength', inplace=True)

In [ ]:
df['type.workout_category'].value_counts()

hiit               41584
strength-cardio    24441
strength            4421
cycling             2424
Yoga                 572
Yoga-HIIT            442
beginner             423
online               256
mobility               8
Name: type.workout_category, dtype: int64

### Save Dataset to File

In [ ]:
# save analysis data to csv file
df.to_csv('/content/drive/My Drive/Freelance/beat81/events_cleaned.csv')

## Data Analysis
In this section, the analysis will be carried out. They will be aggregated on a monthly basis.

### The areas of interest for this analysis are the following:
- session totals (completed/ cancelled)

**For completed sessions:**
- totals for indoor/ outdoor
- totals for workout types
- total available/ used slots
- total no shows
- mean attendance rate
- unique trainers


In [ ]:
# import df
df_loaded = pd.read_csv('/content/drive/My Drive/Freelance/beat81/events_cleaned.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6,11,24,30,31,67,68,69,70,71,72,78,79,88,93,94,95,96,97,98,99,102,126,128,129,130) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Filter to Only Use Necessary Columns & Fix Datatypes

In [ ]:
# filter to only use columns relevant for this analysis
df = df_loaded[['id','date_begin','event_type_id','coach_id','duration','max_participants','participants_count','attendees_count','no_shows_count','waitinglist_count','permitted_cancellations_count','illicit_cancellations_count','is_cancelled','current_status.id','current_status.status_name', 'location.city_code','location.type','type.id','type.workout_category']].copy()
df.rename(columns={"current_status.id": "current_status_id", "current_status.status_name": "current_status_name","type.id":"type_id","type.workout_category":"type_workout_category", "location.type":"location_type","location.city_code":"city"},inplace=True)
df.head()

,id,date_begin,event_type_id,coach_id,duration,max_participants,participants_count,attendees_count,no_shows_count,waitinglist_count,permitted_cancellations_count,illicit_cancellations_count,is_cancelled,current_status_id,current_status_name,city,location_type,type_id,type_workout_category
0,8127571a-53a0-4263-8986-f7e862944057,2017-07-31T16:00:00.000Z,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,7.0,0.0,0.0,0.0,1.0,1.0,False,1c167694-d2d1-4397-b186-e16bbb99d541,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit
1,359335b1-06f2-43ad-b9a3-015166387fdd,2017-07-31T17:00:00.000Z,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,15.0,0.0,0.0,0.0,0.0,1.0,False,10ed15ef-19b1-48f9-a422-29335321e471,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit
2,925de809-bcef-4494-bd9c-f188fb9de82d,2017-08-02T17:00:00.000Z,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,27.0,0.0,0.0,0.0,2.0,1.0,False,e20292d9-6fda-4a9e-bc7f-a79ee54ed556,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit
3,6da81591-0f14-4386-8a21-5efa5c774d02,2017-08-03T17:00:00.000Z,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,17.0,0.0,0.0,0.0,1.0,1.0,False,a3aa6a83-f43a-4a21-aea8-dbdd1a8f2e01,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit
4,9d1fec1a-d4dd-43fd-8882-9d0af41a028f,2017-08-05T09:00:00.000Z,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,14.0,0.0,0.0,0.0,1.0,2.0,False,ccef8c35-d326-470d-a064-e727f5b6fff5,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit


In [ ]:
df.dtypes

id                                object
date_begin                        object
event_type_id                     object
coach_id                          object
duration                           int64
max_participants                   int64
participants_count               float64
attendees_count                  float64
no_shows_count                   float64
waitinglist_count                float64
permitted_cancellations_count    float64
illicit_cancellations_count      float64
is_cancelled                        bool
current_status_id                 object
current_status_name               object
city                              object
location_type                     object
type_id                           object
type_workout_category             object
dtype: object

In [ ]:
# date_begin needs to be a timestamp
df.date_begin = pd.to_datetime(df.date_begin)

In [ ]:
df.dtypes

id                                            object
date_begin                       datetime64[ns, UTC]
event_type_id                                 object
coach_id                                      object
duration                                       int64
max_participants                               int64
participants_count                           float64
attendees_count                              float64
no_shows_count                               float64
waitinglist_count                            float64
permitted_cancellations_count                float64
illicit_cancellations_count                  float64
is_cancelled                                    bool
current_status_id                             object
current_status_name                           object
city                                          object
location_type                                 object
type_id                                       object
type_workout_category                         

In [ ]:
# localize time
df['date_begin_berlin'] = df['date_begin'].dt.tz_convert("Europe/Berlin")
# pd.to_datetime(, unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
df[['date_begin','date_begin_berlin']].head()

,date_begin,date_begin_berlin
0,2017-07-31 16:00:00+00:00,2017-07-31 18:00:00+02:00
1,2017-07-31 17:00:00+00:00,2017-07-31 19:00:00+02:00
2,2017-08-02 17:00:00+00:00,2017-08-02 19:00:00+02:00
3,2017-08-03 17:00:00+00:00,2017-08-03 19:00:00+02:00
4,2017-08-05 09:00:00+00:00,2017-08-05 11:00:00+02:00


### Completed & Cancelled Events

In [ ]:
df['month'] = df.date_begin_berlin.dt.strftime('%Y/%m')
df.head()


,id,date_begin,event_type_id,coach_id,duration,max_participants,participants_count,attendees_count,no_shows_count,waitinglist_count,permitted_cancellations_count,illicit_cancellations_count,is_cancelled,current_status_id,current_status_name,city,location_type,type_id,type_workout_category,date_begin_berlin,month
0,8127571a-53a0-4263-8986-f7e862944057,2017-07-31 16:00:00+00:00,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,7.0,0.0,0.0,0.0,1.0,1.0,False,1c167694-d2d1-4397-b186-e16bbb99d541,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit,2017-07-31 18:00:00+02:00,2017/07
1,359335b1-06f2-43ad-b9a3-015166387fdd,2017-07-31 17:00:00+00:00,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,15.0,0.0,0.0,0.0,0.0,1.0,False,10ed15ef-19b1-48f9-a422-29335321e471,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit,2017-07-31 19:00:00+02:00,2017/07
2,925de809-bcef-4494-bd9c-f188fb9de82d,2017-08-02 17:00:00+00:00,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,27.0,0.0,0.0,0.0,2.0,1.0,False,e20292d9-6fda-4a9e-bc7f-a79ee54ed556,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit,2017-08-02 19:00:00+02:00,2017/08
3,6da81591-0f14-4386-8a21-5efa5c774d02,2017-08-03 17:00:00+00:00,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,17.0,0.0,0.0,0.0,1.0,1.0,False,a3aa6a83-f43a-4a21-aea8-dbdd1a8f2e01,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit,2017-08-03 19:00:00+02:00,2017/08
4,9d1fec1a-d4dd-43fd-8882-9d0af41a028f,2017-08-05 09:00:00+00:00,c77c6678-16ab-4f49-8e4e-66020419abf6,NaN,45,32,14.0,0.0,0.0,0.0,1.0,2.0,False,ccef8c35-d326-470d-a064-e727f5b6fff5,completed,berlin,outdoor,c77c6678-16ab-4f49-8e4e-66020419abf6,hiit,2017-08-05 11:00:00+02:00,2017/08


In [ ]:
df_status = df.groupby(['month','current_status_name','city'])['id'].count().reset_index()

cancelled = df_status.current_status_name == 'cancelled'
completed = df_status.current_status_name == 'completed'

In [ ]:
status_export = df_status[cancelled | completed].pivot_table(index=["current_status_name","city"], columns="month", values="id", aggfunc=np.sum, fill_value=0)
status_export

month                          2017/07  2017/08  ...  2031/04  2031/07
current_status_name city                         ...                  
cancelled           barcelona        0        0  ...        0        0
                    berlin           0        0  ...        1        1
                    cologne          0        0  ...        0        0
                    hamburg          0        0  ...        0        0
                    london           0        0  ...        0        0
                    munich           0        0  ...        0        0
                    vienna           0        0  ...        0        0
completed           barcelona        0        0  ...        0        0
                    berlin           2       22  ...        0        0
                    cologne          0        0  ...        0        0
                    hamburg          0        0  ...        0        0
                    london           0        0  ...        0        0
                    munich           0        0  ...        0        0
                    vienna           0        0  ...        0        0

[14 rows x 119 columns]

In [ ]:
df_result = df_status[cancelled | completed].copy()
df_result['kpi'] = 'Session Total'
df_result.rename(columns={"current_status_name":"variable","id":"count"}, inplace=True)
df_result

,month,variable,city,count,kpi
0,2017/07,completed,berlin,2,Session Total
1,2017/08,completed,berlin,22,Session Total
2,2017/09,completed,berlin,29,Session Total
3,2017/10,completed,berlin,26,Session Total
4,2017/11,completed,berlin,25,Session Total
...,...,...,...,...,...
379,2030/08,cancelled,berlin,1,Session Total
380,2030/10,cancelled,berlin,1,Session Total
381,2031/01,cancelled,berlin,1,Session Total
382,2031/04,cancelled,berlin,1,Session Total


In [ ]:
# status_export.to_csv('/content/drive/My Drive/Freelance/beat81/status_export.csv')

### Stats for Completed Events

In [ ]:
# use only completed events
completed = df.current_status_name == 'completed'
df = df[completed]

In [ ]:
df.current_status_name.value_counts()

completed    17044
Name: current_status_name, dtype: int64

#### Totals In/ Outdoor

In [ ]:
df_indoor = df.groupby(['month','location_type','city'])['id'].count().reset_index()
df_indoor

,month,location_type,city,id
0,2017/07,outdoor,berlin,2
1,2017/08,outdoor,berlin,22
2,2017/09,outdoor,berlin,29
3,2017/10,outdoor,berlin,26
4,2017/11,outdoor,berlin,25
...,...,...,...,...
114,2020/05,outdoor,berlin,479
115,2020/06,indoor,berlin,37
116,2020/06,outdoor,berlin,367
117,2020/06,outdoor,cologne,48


In [ ]:
# indoor_export = df_indoor.pivot_table(index="location_type", columns="month", values="id", aggfunc=np.sum, fill_value=0)
# indoor_export

In [ ]:
# indoor_export.to_csv('/content/drive/My Drive/Freelance/beat81/status_export.csv')

In [ ]:
df_indoor['kpi'] = 'location_type'
df_indoor.rename(columns={"location_type":"variable","id":"count"}, inplace=True)
df_result = df_result.append(df_indoor,ignore_index=True)
df_result

,month,variable,city,count,kpi
0,2017/07,completed,berlin,2,Session Total
1,2017/08,completed,berlin,22,Session Total
2,2017/09,completed,berlin,29,Session Total
3,2017/10,completed,berlin,26,Session Total
4,2017/11,completed,berlin,25,Session Total
...,...,...,...,...,...
422,2020/05,outdoor,berlin,479,location_type
423,2020/06,indoor,berlin,37,location_type
424,2020/06,outdoor,berlin,367,location_type
425,2020/06,outdoor,cologne,48,location_type


#### Totals by Workout Type

In [ ]:
df_wotype = df.groupby(['month','type_workout_category','city'])['id'].count().reset_index()
df_wotype

,month,type_workout_category,city,id
0,2017/07,hiit,berlin,2
1,2017/08,hiit,berlin,22
2,2017/09,hiit,berlin,29
3,2017/10,hiit,berlin,26
4,2017/11,hiit,berlin,25
...,...,...,...,...
178,2020/06,hiit,cologne,22
179,2020/06,hiit,hamburg,5
180,2020/06,strength-cardio,berlin,198
181,2020/06,strength-cardio,cologne,26


In [ ]:
df_wotype['kpi'] = 'workout_category'
df_wotype.rename(columns={"type_workout_category":"variable","id":"count"}, inplace=True)
df_wotype.head()

,month,variable,city,count,kpi
0,2017/07,hiit,berlin,2,workout_category
1,2017/08,hiit,berlin,22,workout_category
2,2017/09,hiit,berlin,29,workout_category
3,2017/10,hiit,berlin,26,workout_category
4,2017/11,hiit,berlin,25,workout_category


In [ ]:
df_result = df_result.append(df_wotype, ignore_index=True)
df_result

,month,variable,city,count,kpi
0,2017/07,completed,berlin,2,Session Total
1,2017/08,completed,berlin,22,Session Total
2,2017/09,completed,berlin,29,Session Total
3,2017/10,completed,berlin,26,Session Total
4,2017/11,completed,berlin,25,Session Total
...,...,...,...,...,...
605,2020/06,hiit,cologne,22,workout_category
606,2020/06,hiit,hamburg,5,workout_category
607,2020/06,strength-cardio,berlin,198,workout_category
608,2020/06,strength-cardio,cologne,26,workout_category


#### Totals by Available Slots, Used Slots, No Shows

In [ ]:
df_attendance = df[['month','max_participants','participants_count','attendees_count','permitted_cancellations_count','illicit_cancellations_count','no_shows_count','waitinglist_count','city']].groupby(['month','city']).sum().reset_index()
df_attendance['kpi'] = 'attendance'
df_attendance

,month,city,max_participants,participants_count,attendees_count,permitted_cancellations_count,illicit_cancellations_count,no_shows_count,waitinglist_count,kpi
0,2017/07,berlin,64,22.0,0.0,1.0,2.0,0.0,0.0,attendance
1,2017/08,berlin,704,384.0,70.0,26.0,18.0,7.0,0.0,attendance
2,2017/09,berlin,928,318.0,237.0,29.0,10.0,39.0,0.0,attendance
3,2017/10,berlin,832,328.0,234.0,12.0,13.0,55.0,0.0,attendance
4,2017/11,berlin,496,340.0,279.0,59.0,59.0,44.0,1.0,attendance
...,...,...,...,...,...,...,...,...,...,...
78,2020/03,munich,384,154.0,117.0,29.0,18.0,4.0,0.0,attendance
79,2020/05,berlin,3353,2882.0,1768.0,1589.0,962.0,54.0,1112.0,attendance
80,2020/06,berlin,4297,3325.0,1984.0,1904.0,1028.0,150.0,709.0,attendance
81,2020/06,cologne,530,323.0,263.0,71.0,55.0,1.0,11.0,attendance


In [ ]:
df_attendance_result = pd.melt(df_attendance, id_vars=['month','kpi','city'], value_vars=['max_participants','participants_count','attendees_count','permitted_cancellations_count','illicit_cancellations_count','no_shows_count','waitinglist_count'], value_name='count')
df_attendance_result

,month,kpi,city,variable,count
0,2017/07,attendance,berlin,max_participants,64.0
1,2017/08,attendance,berlin,max_participants,704.0
2,2017/09,attendance,berlin,max_participants,928.0
3,2017/10,attendance,berlin,max_participants,832.0
4,2017/11,attendance,berlin,max_participants,496.0
...,...,...,...,...,...
576,2020/03,attendance,munich,waitinglist_count,0.0
577,2020/05,attendance,berlin,waitinglist_count,1112.0
578,2020/06,attendance,berlin,waitinglist_count,709.0
579,2020/06,attendance,cologne,waitinglist_count,11.0


In [ ]:
df_result = df_result.append(df_attendance_result,ignore_index=True)

In [ ]:
df_result

,month,variable,city,count,kpi
0,2017/07,completed,berlin,2.0,Session Total
1,2017/08,completed,berlin,22.0,Session Total
2,2017/09,completed,berlin,29.0,Session Total
3,2017/10,completed,berlin,26.0,Session Total
4,2017/11,completed,berlin,25.0,Session Total
...,...,...,...,...,...
1186,2020/03,waitinglist_count,munich,0.0,attendance
1187,2020/05,waitinglist_count,berlin,1112.0,attendance
1188,2020/06,waitinglist_count,berlin,709.0,attendance
1189,2020/06,waitinglist_count,cologne,11.0,attendance


#### Unique Trainers

In [ ]:
df_trainers = df[['month','coach_id','city']]

df_trainers.shape

(17044, 3)

In [ ]:
df_trainers_result = df_trainers.drop_duplicates()
df_trainers_result.shape

(1008, 3)

In [ ]:
df_trainers_agg = df_trainers_result.groupby(['month','city']).count().reset_index()
df_trainers_agg.rename(columns={"coach_id":"count"}, inplace=True)
df_trainers_agg['kpi'] = 'trainers'
df_trainers_agg['variable'] = 'unique trainers'
df_trainers_agg

,month,city,count,kpi,variable
0,2017/07,berlin,0,trainers,unique trainers
1,2017/08,berlin,0,trainers,unique trainers
2,2017/09,berlin,0,trainers,unique trainers
3,2017/10,berlin,0,trainers,unique trainers
4,2017/11,berlin,1,trainers,unique trainers
...,...,...,...,...,...
78,2020/03,munich,5,trainers,unique trainers
79,2020/05,berlin,33,trainers,unique trainers
80,2020/06,berlin,35,trainers,unique trainers
81,2020/06,cologne,6,trainers,unique trainers


In [ ]:
df_result = df_result.append(df_trainers_agg, ignore_index=True)

### Summary

In [ ]:
df_result['count'] = df_result['count'].astype(int)

In [ ]:
df_result.pivot_table(index=["kpi","variable","city"], columns="month", values=["count"], aggfunc=np.sum, fill_value=0)

count          ...                
month                                      2017/07 2017/08  ... 2031/04 2031/07
kpi              variable        city                       ...                
Session Total    cancelled       barcelona       0       0  ...       0       0
                                 berlin          0       0  ...       1       1
                                 cologne         0       0  ...       0       0
                                 hamburg         0       0  ...       0       0
                                 london          0       0  ...       0       0
...                                            ...     ...  ...     ...     ...
workout_category strength-cardio barcelona       0       0  ...       0       0
                                 berlin          0       0  ...       0       0
                                 cologne         0       0  ...       0       0
                                 hamburg         0       0  ...       0       0
                                 munich          0       0  ...       0       0

[100 rows x 119 columns]

In [ ]:
df_result

,month,variable,city,count,kpi
0,2017/07,completed,berlin,2,Session Total
1,2017/08,completed,berlin,22,Session Total
2,2017/09,completed,berlin,29,Session Total
3,2017/10,completed,berlin,26,Session Total
4,2017/11,completed,berlin,25,Session Total
...,...,...,...,...,...
1269,2020/03,unique trainers,munich,5,trainers
1270,2020/05,unique trainers,berlin,33,trainers
1271,2020/06,unique trainers,berlin,35,trainers
1272,2020/06,unique trainers,cologne,6,trainers


In [ ]:
df_result.to_csv('/content/drive/My Drive/Freelance/beat81/huesch_export.csv',index=False)